# Question Answering on a Knowledge Graph

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial10_Knowledge_Graph.ipynb)

Haystack allows storing and querying knowledge graphs with the help of pre-trained models that translate natural language questions to SPARQL queries..
This tutorial demonstrates how to load an existing knowledge graph into haystack, load a pre-trained retriever, and execute questions on the knowledge graph.
The training of models that translate natural language questions into SPARQL queries is currently not supported.

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git

In [ ]:
# Here are some imports that we'll need

import subprocess
import time
from pathlib import Path

from farm.utils import initialize_device_settings

from haystack.graph_retriever.kgqa import Text2SparqlRetriever
from haystack.knowledge_graph.graphdb import GraphDBKnowledgeGraph


## Launching a GraphDB instance

In [ ]:
# Unfortunately, there seems to be no good way to run GraphDB in colab environments
# Start a GraphDB server with docker
print("Starting GraphDB ...")
status = subprocess.run(
    ['docker run -d -p 7200:7200 --name graphdb-instance-lcquad docker-registry.ontotext.com/graphdb-free:9.4.1-adoptopenjdk11'], shell=True
)
if status.returncode:
    raise Exception("Failed to launch GraphDB. Maybe it is already running?")
time.sleep(30)

## Creating a new repository (also known as document index in haystack's document stores)

In [ ]:
# Initialize a new GraphDBKnowledgeGraph store, create an index and load triples into the store

kg = GraphDBKnowledgeGraph()
kg.create_index(config_path=Path("../data/repo-config.ttl"))
kg.index = "lcquad_full_wikidata"
kg.import_from_ttl_file(index="lcquad_full_wikidata", path=Path("../data/tutorial10_knowledge_graph.ttl"))


In [ ]:
# here are ten example triples

print(kg.get_all_triples()[:10])

In [ ]:
## Initialize Retriever, which translates questions from natural language text to SPARQL queries
device, n_gpu = initialize_device_settings(use_cuda=True)
kgqa_retriever = Text2SparqlRetriever(knowledge_graph=kg, model_name_or_path="../saved_models/lcquad_full_wikidata", top_k=1)

## Query Execution

We can now ask question that will be answered by our knowledge graph!
One limitation though: our pre-trained model can only generate questions about entities it has seen during training.
Otherwise it cannot translate the name of the entity to the identifier used in the knowledge graph.

In [ ]:
result = kgqa_retriever.retrieve(question_text="What periodical literature does Delta Air Lines use as a moutpiece?")
## SPARQL query: select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 }
## Answer: wd:Q3486420